**Sentinel allows customers to implement policy-as-code in the same way that Terraform implements infrastructure-as-code.**

**The Sentinel Command Line Interface (CLI) allows you to apply and test Sentinel policies including those that use mocks generated from Terraform Cloud and Terraform Enterprise.**

**TFC Free plans support 1 Policy Set and 5 Policies: https://www.hashicorp.com/products/terraform/pricing**

# Where is Sentinel Used in Terraform?

* Sentinel policies are checked between the standard plan and apply steps of Terraform runs.

![Alt text](image-20.png)

* Policies have different enforcement levels: <u>advisory</u>, <u>soft-mandatory</u>, and <u>hard-mandatory</u>.
  * Advisory
    - Only logs violations
  * Soft Mandatory
    - Can be overridden by authorized users:
      - Members of the owners team
      - Members of teams with the "Manage Policy Overrides" permission
  * Hard Mandatory
    - Cannot be overridden by anyone
  
> Customers often create new Sentinel policies as Advisory, then transition to Soft Mandatory, and eventually to Hard Mandatory.

* Violations prevent runs from being applied unless a user with sufficient authority overrides them.
* Sentinel policies can evaluate the attributes (arguments and exported attributes) of existing and new resources and data sources based on information from the current run:
  * the **plan**,
  * the **configuration**,
  * the **current state**, 
  * and other **run data** including cost estimates
* This ensures that resources comply with all policies before they are provisioned.

# Types of Terraform Sentinel Policies
* There are essentially four types of Terraform Sentinel policies corresponding to the 4 Terraform Sentinel imports:
  * Policies can use the **tfplan/v2** import to restrict specific attributes of specific resources and data sources in the current Terraform plan.
  * Policies can use the **tfconfig/v2** import to restrict the configuration of Terraform modules, variables, resources, data sources, providers, provisioners, and outputs.
  * Policies can use the **tfstate/v2** import to check whether previously provisioned resources or data sources have attributes with values that are no longer allowed.
  * Policies can use the **tfrun** import to check workspace and run metadata and whether cost estimates for planned resources are within limits.
* Some policies might use more than one of these imports.

# 1. Create TFC Workspace associated to VCS repo

The first thing we are going to do in this exercise is to sync a VCS repo containig Terraform code with Terraform Cloud. To that end, we are going to use a local terraform workspace to create the TFC Workspace that will create infrastructure in AWS and against which we are going to test our Sentinel policies.

## 1.1 Inputs
### **variables.tfvars**
| Variable Name   | Description              |
| --------------- | ------------------------ |
| organization    | TFC Organization name    |
| project         | Project name             |
| gh_token        | Github OAuth Token       |
| repo            | Github repo              |
| working_dir     | Relative path where TF code is placed | 
| workspace       | Workspace Name |

In [1]:
%%bash
cd tf-config-tfe
terraform init
terraform apply -var-file=variables.tfvars -auto-approve


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/tfe from the dependency lock file
- Using previously-installed hashicorp/tfe v0.48.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # tfe_project.sentinel_test_project will be created
  + resource "tfe_project" "sentinel_test_project" {
      + id           = (known after apply)
      + name         = "Se

![Alt text](image.png)
![Alt text](image-1.png)

Now that we have a working Terraform configuration and workspace, lets create a policy to set guardrails around one of type of resources created by our config. We want to enforce 2 organization requirements for S3 buckets:

* All S3 buckets should have `department` and `environment` tags
* All S3 bucket should use the `private` ACL to prevent accidental data leaks

# 2. Sentinel Imports

A Sentinel policy can include imports which enable a policy to access reusable libraries, external data and functions. Terraform Cloud provides four built-in imports that can be used for a policy check:

| Import  | Description |
| ------- | ----------- |
| [tfplan](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfplan-v2)  |	provides access to Terraform plan details which represent the changes Terraform will make to the desired state |
| [tfconfig](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfconfig-v2) |	provides access to Terraform configuration that is being used to describe the desired state |
| [tfstate](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfstate-v2) |	provides access to Terraform statewhich represents what Terraform knows about the real world resources |
| [tfrun](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfrun) |	provides access to information about a run |

Note: Some imports can have a v2 suffix which indicates they represent the new data structures used post Terraform 0.12

We have created an example sentinel policy in `restrict-s3-buckets.sentinel`. Let's analyze each of its parts:
```hcl

import "tfplan/v2" as tfplan

# Filter S3 buckets
s3_buckets = filter tfplan.resource_changes as address, rc {
  rc.type is "aws_s3_bucket" and
  (rc.change.actions contains "create" or rc.change.actions is ["update"])
}
```
* `tfplan/v2` is a frequently used import in policies since it provides details about planned changes. Later in this lab we will go over how you can determine what information is available to your policy from the import.
* `tfplan.resource_changes` is a collection with the resource address as the key and a resource change object as the value.
We are iterating over each of the resource change objects in the collection and using the filter function to filter out change objects whose type is `aws_s3_bucket`.
* The `type` name matches the resource block we would define in a .tf file to manage S3 buckets.

To understand where is that what we are checking let's run a `Plan` on the TFC workspace and get some `mock` data.

In [2]:
%%bash
# Add AWS creds to your workspace
# SE only
doormat login 
doormat aws --account aws_jose.merchan_test --tf-push --tf-workspace  Sentinel --tf-organization josemerchan-training

time="2023-10-05T19:39:01+02:00" level=info msg="current session expires on 2023-10-05 21:11:47 +0200 CEST\n"
time="2023-10-05T19:39:01+02:00" level=info msg="run `doormat login -f` to refresh existing doormat credentials"
time="2023-10-05T19:39:01+02:00" level=warning msg="The --tf-push flag is being deprecated and has been replaced with a command. In the future, run 'doormat aws tf-push'"
time="2023-10-05T19:39:02+02:00" level=info msg="Getting Creds for TFC/TFE"
time="2023-10-05T19:39:05+02:00" level=info msg="Creds pushed to \"Sentinel\""
time="2023-10-05T19:39:05+02:00" level=info msg=Done


## 2.1 Let's Create a Plan-only run


![Alt text](image-2.png)

A plan will be queued. Once finished it will give use what is planned to be changed. Additionally it gives us the option to `Download Sentinel mocks` 

![Alt text](image-3.png)

You can download them or simply use the sample mocks provided within this repo, within the /S3-demo/mocks directory. The one we care about at this point is `mock-tfplan-v2.sentinel`

The file contains a number of top level objects (`collections`) that can be iterated

| file example             | doc reference            |
| -----------------------  | -------------            |
| ![Alt text](image-4.png) | ![Alt text](image-5.png) |

Sentinel imports are structured as a series of collections with a number of attributes, documented as in the image above (right)

So back to our example we are iterating over resource changes. If you look at the plan output above there are 6 resources that "will be created". Those 6 all also the resources within the resource_changes list

![Alt text](image-6.png)


```hcl

import "tfplan/v2" as tfplan

# Filter S3 buckets
s3_buckets = filter tfplan.resource_changes as address, rc {
  rc.type is "aws_s3_bucket" and
  (rc.change.actions contains "create" or rc.change.actions is ["update"])
}
```

* From the collection the only one that matches is the first resource: `aws_s3_bucket.dev`
* The other thing we are doing is apply and `AND` and verify that the kind of change apply to the resource in this case whether the resource is `create`(d) or `update`(d)

| ![Alt text](image-7.png) |  ![Alt text](image-8.png) |

The resource `aws_s3_bucket.dev` matches both conditions. The result is the creation of a list (or based on the [documentation](https://docs.hashicorp.com/sentinel/language/collection-operations#filter-expression):  `a subset of the provided collection` ) with a single element on it

## 2.2 Rules

The next steps is check that our collection (the subset of it based on the `filter` expression) matches the characteristics we want to enforce:
* All S3 buckets should have `department` and `environment` tags
* All S3 bucket should use the `private` ACL to prevent accidental data leaks

1. Let's focus on the first of those. 

```hcl
# The tags we want to enforce
required_tags = ["department", "environment"]

# Lets create a subset/list if it contains tags but those tags are not in the "required_tags" list
tag_violators = filter s3_buckets as address, bucket {
  any required_tags as rtag {
    rtag not in bucket.change.after.tags
  }
}
# Given a list of aws_s3_bucket resources with invalid tags, validate if the list is empty.
# If not empty the rule evaluate to 'false' (fail), but if empty then evaluate to 'true' (pass)
bucket_should_have_required_tags = rule {
  tag_violators is empty
}
```

> We are using the `any` expression to test if any of the required tags are not present in the bucket's list of tags after changes are applied. If a tag is found to be missing, the expression evaluates to true and the resource is added to the violators list.

Let's go back to our file in `S3-demo/mocks/mock-tfplan-v2.sentinel`. 

The resource has the `tag` **environment** but not **department**. For this reason this [rule](https://docs.hashicorp.com/sentinel/intro/getting-started/rules) will result in a `FAIL` when evaluated.



```json
	"aws_s3_bucket.dev": {
		"address": "aws_s3_bucket.dev",
		"change": {
			"actions": [
				"create",                             # action create
			],
			"after": {
				"acl":                       "private",
				"bucket_prefix":             null,
				"cors_rule":                 [],
				"force_destroy":             true,
				"grant":                     [],
				"lifecycle_rule":            [],
				"logging":                   [],
				"policy":                    null,
				"replication_configuration": [],
				"tags": {
					"environment": "dev",                # Just enviroment, department is missing
				},
				"tags_all": {
					"environment": "dev",
				},
				"website": [],
			},
			"after_unknown": {
				...
			},
			"before": null,
		},
		"deposed":        "",
		"index":          null,
		"mode":           "managed",
		"module_address": "",
		"name":           "dev",
		"provider_name":  "registry.terraform.io/hashicorp/aws",
		"type":           "aws_s3_bucket",             # aws_s3_bucket resource
	},

```


1. Let's move onto the second one, making sure that the acl policy associated to the bucket is set to `private`

```bash
# Given the collection of resources in the tfplan iterate and obtain those whose type is aws_s3_bucket_acl and that are going to be created or updated
s3_bucket_acls = filter tfplan.resource_changes as address, rc {
  rc.type is "aws_s3_bucket_acl" and
  (rc.change.actions contains "create" or rc.change.actions is ["update"])
}

# For the collection of aws_s3_bucket_acl get those whose change.after.acl is not private
acl_violators = filter s3_bucket_acls as address, bucket {
  bucket.change.after.acl != "private"
}
# If the subset above is not empty then result in FAIL
bucket_acl_should_be_private = rule {
  acl_violators is empty
}
```

Again, let's go back to our file in `S3-demo/mocks/mock-tfplan-v2.sentinel`. Here we can find a resource that matches both conditions

```json
	"aws_s3_bucket_acl.dev": {
		"address": "aws_s3_bucket_acl.dev",
		"change": {
			"actions": [
				"create",                      # action create
			],
			"after": {
				"acl": "public-read",           # is not private
				"expected_bucket_owner": null,
			},
			"after_unknown": {
				"access_control_policy": true,
				"bucket":                true,
				"id":                    true,
			},
			"before": null,
		},
		"deposed":        "",
		"index":          null,
		"mode":           "managed",
		"module_address": "",
		"name":           "dev",
		"provider_name":  "registry.terraform.io/hashicorp/aws",
		"type":           "aws_s3_bucket_acl",  # aws_s3_bucket_acl
	},
````


## 2.3 main rule
Each Sentinel policy is expected to contain a main rule. The result of the policy depends on the evaluated contents of the main rule. For booleans, a policy passes on a true value, and fails on a false value.

Let's add a main rule, the result of which is the combination of the 2 rules we have defined earlier. Note that using Sentinel we are able to apply our policy logic to multiple types of resources (`aws_s3_bucket_acl` and `aws_s3_bucket`)and combine the results.

```hcl
main = rule {
  bucket_should_have_required_tags and
  bucket_acl_should_be_private
}
```
We are doing a logical `AND` with our 2 rules. If either one of our rules evaluates to false, our main rule evaluates to false and our policy check fails.

## 2.4 Testing

We have created a Sentinel policy and based on what we know based on the mock Sentinel info, the infrastructure is not in compliance with the policy. Before applying the policy via `Policy Set` in TFC is advisable to run local testing (or via CI/CD) to make sure our Sentinel policies are properly written.

The Sentinel CLI allows for the development and testing of policies outside of TFC/TFE. Sentinel Mocks are imports used to mock the data available to the Sentinel engine when its runs after a plan operation in TFE/TFC. To that end we are going to use the mock data we can download from TFC, which we have made available in `S3-demo/mocks`
* https://docs.hashicorp.com/sentinel/intro/getting-started/install

When you download the mock data from Terraform Cloud you get a tar.gz. Within the available files you get a `sentinel.hcl`. In this case what we have done is move and rename that file as `sentinel-mocks.hcl` and placed in the `S3-demo` directory. Finally, we have modify the relative path of the import to refer to the in `mocks/....`

| Original                              | Modified |
| --------                              | -------- |
| ![Original](image-11.png)             | ![Modified](image-10.png) |

For Sentinel to use to use mocks, the CLI must be provided with a configuration file. This can be specified using the -config=path flag. This configuration file is the  `sentinel-mocks.hcl` we have just created, which simply defines where to obtain the `imports`

Let's run our first local policy test using the Sentinel CLI:

In [16]:
! sentinel apply -config=sentinel-mocks.hcl restrict-s3-buckets.sentinel


Execution trace. The information below will show the values of all
the rules evaluated. Note that some rules may be missing if
short-circuit logic was taken.

Note that for collection types and long strings, output may be
truncated; re-run "sentinel apply" with the -json flag to see the
full contents of these values.

The trace is displayed due to a failed policy.

Fail - restrict-s3-buckets.sentinel

Description:
  A sentinel policy for S3 buckets that enforces required tags are provided
  and bucket acl is set to private

Print messages:

aws_s3_bucket.dev actual tags: ["environment"] 
	required tags: ["department" "environment"] 

aws_s3_bucket_acl.dev actual acl: public-read 
	required acl: private 


restrict-s3-buckets.sentinel:50:1 - Rule "main"
  Value:
    false

restrict-s3-buckets.sentinel:20:1 - Rule "bucket_should_have_required_tags"
  Value:
    false



You should see a failure message indicating the main rule failed as well as the nested rule that resulted in the failure.

> Sentinel uses lazy evaluation, since the first rule evaluated to false, the 2nd one was not evaluated fully because Sentinel knows the policy has already failed.

## 2.5 Logging
Unlike HCL, Sentinel support print functions that can be used to enrich the information provided.

Uncomment
```hcl
# Main rule that requires other rules to be true
/*
# Before the main rule!
for tag_violators as name, bucket {
  print(bucket.address, "actual tags:", keys(bucket.change.after.tags), "\n\trequired tags:", required_tags,"\n")
}

for acl_violators as name, bucket {
  print(bucket.address, "actual acl:", bucket.change.after.acl, "\n\trequired acl:", "private","\n")
}
*/
```

In [15]:
! sentinel apply -config=sentinel-mocks.hcl restrict-s3-buckets.sentinel


Execution trace. The information below will show the values of all
the rules evaluated. Note that some rules may be missing if
short-circuit logic was taken.

Note that for collection types and long strings, output may be
truncated; re-run "sentinel apply" with the -json flag to see the
full contents of these values.

The trace is displayed due to a failed policy.

Fail - restrict-s3-buckets.sentinel

Description:
  A sentinel policy for S3 buckets that enforces required tags are provided
  and bucket acl is set to private

Print messages:

aws_s3_bucket.dev actual tags: ["environment"] 
	required tags: ["department" "environment"] 

aws_s3_bucket_acl.dev actual acl: public-read 
	required acl: private 


restrict-s3-buckets.sentinel:50:1 - Rule "main"
  Value:
    false

restrict-s3-buckets.sentinel:20:1 - Rule "bucket_should_have_required_tags"
  Value:
    false



# 3. Unit Testing

While running apply is helpful to validate a policy, Sentinel comes with a **built-in** test framework to validate a policy behaves as expected for a number of cases.

Sentinel is opinionated about the folder structure required for tests. This opinionated structure allows testing to be as simple as running a single sentinel test command
The structure Sentinel expects for a policy test is `test/<policy>/<test_name>.[hcl|json]` where <policy> is the name of your policy file without the file extension.

Within that folder should be a list of HCL or JSON files. Each representing a single test case.

We have created two test cases within `S3-demo/test/restrict-s3-buckets/[fail.hcl | pass.hcl]` 


The tests import mocked data and based on that data the expected result. For the `fail.hcl` we are using the mocked data we have been using so far, since we know it fails.

```hcl
mock "tfplan/v2" {
  module {
    source = "../../mocks/mock-tfplan-v2.sentinel" # Relative path to the location of the mock data
  }
}

test {
  rules = {
    main = false # Expected result
  }
}

```

For a mock data that would pass the test what we have done is to copy the `mock-tfplan-v2.sentinel` into `mock-tfplan-v2-pass.sentinel` and modify the mock data accordingly


| Resource            | `mock-tfplan-v2.sentinel` | `mock-tfplan-v2-pass.sentinel`|
|-------------------- | -----------------------   | ----------------------------- |
| `aws_s3_bucket`     | ![FAIL](image-13.png)     |   ![PASS](image-14.png)       |
| `aws_s3_bucket_acl` | ![FAIL](image-16.png)     |   ![PASSS](image-15.png)      |

Our `pass.hcl` looks as follow
```hcl
mock "tfplan/v2" {
  module {
    source = "../../mocks/mock-tfplan-v2-pass.sentinel" # Relative path to pass mock data
  }
}

test {
  rules = {
    main = true                                        # Expected result
  }
}
```

Run sentinel test in the directory with your sentinel policy to verify your config works! The verbose parameter displays output from print statements which can be useful for debugging.
```bash
sentinel test --verbose
```


> Sentinel tests can be integrated in CI pipelines to ensure policy updates continue to have the intended effect. When teams discover additional use cases/exceptions, these are added into policies and corresponding test cases created. This allows policy updates to be made with confidence.

## 3.1 Results

In [18]:
! sentinel test --verbose

Installing test modules for test/restrict-s3-buckets/fail.hcl
Installing test modules for test/restrict-s3-buckets/pass.hcl

PASS - restrict-s3-buckets.sentinel
  PASS - test/restrict-s3-buckets/fail.hcl


    logs:
      aws_s3_bucket.dev actual tags: ["environment"]
      	required tags: ["department" "environment"]

      aws_s3_bucket_acl.dev actual acl: public-read
      	required acl: private
    trace:
      restrict-s3-buckets.sentinel:50:1 - Rule "main"
        Value:
          false

      restrict-s3-buckets.sentinel:20:1 - Rule "bucket_should_have_required_tags"
        Value:
          false
  PASS - test/restrict-s3-buckets/pass.hcl

    trace:
      restrict-s3-buckets.sentinel:50:1 - Rule "main"
        Value:
          true

      restrict-s3-buckets.sentinel:33:1 - Rule "bucket_acl_should_be_private"
        Value:
          true

      restrict-s3-buckets.sentinel:20:1 - Rule "bucket_should_have_required_tags"
        Value:
          true
1 tests completed in 7.9202

# 4. Attach Sentinel Policy to Workspace

Now that we are satisfy with our Policy (as Code) we can associate it to our workspaces. To that end we need to create a policy set. Policy sets can be feed sentinel policies via:
* VCS integration
* CLI
* API (`tfe` provider included)

![Policy Set Creation](image-17.png)

In this case what we are going to do is to attach the policy via TFE. To that end we need to:
1. Create a directory where we are going to copy our `restrict-s3-buckets.sentinel` policy. The directory has already been created and is named **policy-set** (`/S3-demo/policy-set`)
2. On that directory we are going to create a `sentinel.hcl` file that defines the name of the policy, its `source` code and `enforcement_level``

```hcl
policy "restrict-s3-buckets" {
  source            = "./restrict-s3-buckets.sentinel"
  enforcement_level = "hard-mandatory"
}
```

3. In the directory `/S3-demo/tf-config-tfe/main.tf` we are going to remove the multiline comments

```hcl

data "tfe_slug" "test" {
  // point to the local directory where the policies are located.
  source_path = "../policy-set"
}

resource "tfe_policy_set" "test" {
  name         = "aws-s3-policy-control"
  description  = "Tags and ACL controls"
  organization = var.organization
  # global       = true  
  workspace_ids = [tfe_workspace.sentinel_test_workspace.id]

  // reference the tfe_slug data source.
  slug = data.tfe_slug.test
}

```
4. Re-apply the terraform Configuration


In [19]:
%%bash
cd tf-config-tfe
terraform init
terraform apply -var-file=variables.tfvars -auto-approve



Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/tfe from the dependency lock file
- Using previously-installed hashicorp/tfe v0.48.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
data.tfe_slug.test: Reading...
tfe_project.sentinel_test_project: Refreshing state... [id=prj-ReVdxfyKRWqi4HJV]
data.tfe_slug.test: Read complete after 0s [id=eab438d24b8498fc576384681e4d56b76d1af05b7ac61160370ad20f0443991b]
tfe_workspace.sentinel_test_workspace: Refreshing state... [id=ws-342p6v9hqAGKBwiW]

Terraform used the selected providers to generate the following e

## 4.1. Attempt an apply on our TFC Sentinel Workspaces

As you can expect the apply does not run because our policy fails and it's set to `hard-mandatory`

![Alt text](image-18.png)

# 5. Clean Up

In [21]:
%%bash
cd tf-config-tfe
terraform destroy -auto-approve -var-file=variables.tfvars

tfe_policy_set.test: Refreshing state... [id=polset-Zz2vSAwGFD2wX8MM]
tfe_project.sentinel_test_project: Refreshing state... [id=prj-ReVdxfyKRWqi4HJV]
tfe_workspace.sentinel_test_workspace: Refreshing state... [id=ws-342p6v9hqAGKBwiW]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # tfe_policy_set.test will be destroyed
  - resource "tfe_policy_set" "test" {
      - description   = "Tags and ACL controls" -> null
      - global        = false -> null
      - id            = "polset-Zz2vSAwGFD2wX8MM" -> null
      - kind          = "sentinel" -> null
      - name          = "aws-s3-policy-control" -> null
      - organization  = "josemerchan-training" -> null
      - overridable   = false -> null
      - policy_ids    = [] -> null
      - slug          = {
          - "id"          = "eab438d24b8498fc576384681e4d56b76d1af05b7ac6116037